# Mean Reversion Strategy on Technology Stocks

In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt  
%matplotlib inline
plt.style.use('seaborn-darkgrid')

In [2]:
# Fix the backtesting period

end = pd.datetime.now().date()
start = end - pd.Timedelta(days=1*252)

<ipython-input-2-547ee0215d57>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  end = pd.datetime.now().date()


In [3]:
# Create a class for backtesting pairs
class mean_reversion_pairs:
    
    # Create object attributes/methods
    def __init__(self,x,y,start,end,lookback,std_dev):
        
        self.x = x # Independent variable
        self.y = y # Dependent variable
        self.start =start 
        self.end = end
        self.lookback = lookback # Lookback period to consider moving average
        self.std_dev = std_dev # Number of standard deivations away to create bollinger band
        
        self.fetch_data()
        self.hedge_ratio()
        self.ADFtest()
        self.indicators()
        self.positions()
        self.returns()
        self.calc_drawdown()
        
    # To fetch the data from yahoo finance for both stocks x and y
    def fetch_data(self):
        
        self.dfx = yf.download(self.x,start,end)
        self.dfy = yf.download(self.y,start,end)
        
        # Create a single dataframe to concat only the closing prices for backtesting purpose
        self.df = pd.concat([self.dfx['Adj Close'],self.dfy['Adj Close']],axis=1)
        self.df.columns =[self.x,self.y]
        self.df.index = pd.to_datetime(self.df.index)
    
    # Calculate hedge ratio by running ordinary least square regression
    def hedge_ratio(self):
        import statsmodels.api as sm
        self.model = sm.OLS(self.df[self.y].iloc[:120],self.df[self.x].iloc[:120])
        self.model = self.model.fit()
        self.df['spread'] = self.df[self.y] - self.model.params[0] * self.df[self.x]
        return self.model.params[0]
    
    # Perform ADF test on the spread of the pair and return the T statistic value
    def ADFtest(self):
        from statsmodels.tsa.stattools import adfuller
        self.ADF = adfuller(self.df['spread'],maxlag=1)
        return self.ADF[0]
    
    # Compute the required indicators for bollinger band
    def indicators(self):
        
        # Moving Average
        self.df['moving_average'] = self.df['spread'].rolling(self.lookback,center=False).mean()
        
        # Moving Standard Deviation
        self.df['moving_std_dev'] = self.df['spread'].rolling(self.lookback,center=False).std()

        # Upper band and lower band
        self.df['upper_band'] = self.df['moving_average'] + self.std_dev*self.df['moving_std_dev']
        self.df['lower_band'] = self.df['moving_average'] - self.std_dev*self.df['moving_std_dev']
    
    # Compute the positions based on the signal
    def positions(self):
        
        # Long positions
        self.df['long_entry'] = self.df['spread'] < self.df['lower_band']
        self.df['long_exit'] = self.df['spread'] >= self.df['moving_average']

        self.df['positions_long'] = np.nan
        self.df.loc[self.df['long_entry'], 'positions_long'] = 1
        self.df.loc[self.df['long_exit'], 'positions_long'] = 0
        self.df['positions_long'] = self.df['positions_long'].fillna(method='ffill')

        # Short positions
        self.df['short_entry'] = self.df['spread'] > self.df['upper_band']
        self.df['short_exit'] = self.df['spread'] <= self.df['moving_average']

        self.df['positions_short'] = np.nan
        self.df.loc[self.df['short_entry'], 'positions_short'] = -1
        self.df.loc[self.df['short_exit'], 'positions_short'] = 0
        self.df['positions_short'] = self.df['positions_short'].fillna(method='ffill')
        
        # Positions
        self.df['positions'] = self.df['positions_long'] + self.df['positions_short']
    
    # Calculate the strategy returns
    def returns(self):
        
        self.df['percentage_change'] = (self.df['spread'] - self.df['spread'].shift(1))/(self.model.params[0]*self.df[self.x] + self.df[self.y])
        self.df['strategy_returns'] = self.df['positions'].shift(1) * self.df['percentage_change']
        self.df['cumulative_returns'] = (self.df['strategy_returns']+1).cumprod()
        print("The total strategy returns are " ,((self.df['cumulative_returns'].iloc[-1]-1)*100))
        return (self.df['cumulative_returns'].iloc[-1]-1)*100
    
    # Calculate the max drawdown of the returns
    def calc_drawdown(self):
        # Calculate the running maximum
        self.running_max = np.maximum.accumulate(self.df['cumulative_returns'].dropna())
        # Ensure the value never drops below 1
        self.running_max[self.running_max < 1] = 1
        # Calculate the percentage drawdown
        self.drawdown = (self.df['cumulative_returns'])/self.running_max - 1
        return self.drawdown.min()*100

In [4]:
# Run the strategy by considering different Technology stocks of same cap with different look back period and number of standard deviations away from the mean
Stocks_X_list = ['TCS.NS','INFY.NS','HCLTECH.NS','WIPRO.NS','TECHM.NS','LTI.NS','MINDTREE.NS']
Stocks_Y_list = ['TCS.NS','INFY.NS','HCLTECH.NS','WIPRO.NS','TECHM.NS','LTI.NS','MINDTREE.NS']

X=[]
Y=[]

lookback_list =range(5,25,5)
stdev_list = [1,1.5,2,2.5]

lookbackperiod =[]
stdev_range = []
Hedge_ratio = []
ADF_Tstat = []
Strat_Returns = []
Drawdown = []

for i in Stocks_X_list:
    for j in Stocks_Y_list:
        for k in lookback_list:
            for l in stdev_list:
                if i == j:
                    continue
                else:
                
            
                    Pairs = mean_reversion_pairs(i,j,start,end,k,l)
                    X.append(i)
                    Y.append(j)
                    lookbackperiod.append(k)
                    stdev_range.append(l)
                    Hedge_ratio.append(Pairs.hedge_ratio())
                    ADF_Tstat.append(Pairs.ADFtest())
                    Strat_Returns.append(Pairs.returns())
                    Drawdown.append(Pairs.calc_drawdown())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -10.944519174982414
The total strategy returns are  -10.944519174982414
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -10.56694842531366
The total strategy returns are  -10.56694842531366
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%***

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -18.804914881754854
The total strategy returns are  -18.804914881754854
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -20.28600869256025
The total strategy returns are  -20.28600869256025
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -15.058652149180729
The total strategy returns are  -15.058652149180729
[*********************100%***********************]  1 of 1 complet

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -17.454843236445516
The total strategy returns are  -17.454843236445516
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  2.66910590547067
The total strategy returns are  2.66910590547067
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -22.170493957278246
The total strategy returns are  -22.170493957278246
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -18.629195096121354
The total strategy returns are  -18.629195096121354
[*********************100%************

The total strategy returns are  -12.051000677876711
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -10.143410490453874
The total strategy returns are  -10.143410490453874
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -5.810514871468553
The total strategy returns are  -5.810514871468553
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -6.001347202932383
The total strategy returns are  -6.001347202932383
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -5.8463997210442376
The total strategy returns are  -5.8

[*********************100%***********************]  1 of 1 completed
The total strategy returns are  3.2827034649164366
The total strategy returns are  3.2827034649164366
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -2.405460793421399
The total strategy returns are  -2.405460793421399
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  1.0672565367476405
The total strategy returns are  1.0672565367476405
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  3.341504802375983
The total strategy returns are  3.341504802375983
[*********************100%***********************]  1 of 1 completed
[*********************100%****************

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -5.650970234854203
The total strategy returns are  -5.650970234854203
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -0.6574234656606004
The total strategy returns are  -0.6574234656606004
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.7050786303308465
The total strategy returns are  0.7050786303308465
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -4.745403376839297
The total strategy returns are  -4.745403376839297
[*********************100%************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  7.786207623006325
The total strategy returns are  7.786207623006325
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  4.916919459028435
The total strategy returns are  4.916919459028435
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  10.117694292807267
The total strategy returns are  10.117694292807267
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  3.3325051514389914
The total strategy returns are  3.3325051514389914
[*********************100%******************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -3.741967870301943
The total strategy returns are  -3.741967870301943
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -12.903118475721765
The total strategy returns are  -12.903118475721765
[*********************100%***********************]  1 of 1 completed
[*********************100%***

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  5.9154392361362
The total strategy returns are  5.9154392361362
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  2.0790151073822427
The total strategy returns are  2.0790151073822427
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -0.4997662271963521
The total strategy returns are  -0.4997662271963521
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  1.6346361457504122
The total strategy returns are  1.6346361457504122
[*********************100%******************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  4.650591639588497
The total strategy returns are  4.650591639588497
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  14.241294385004299
The total strategy returns are  14.241294385004299
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  3.272172087283498
The total strategy returns are  3.272172087283498
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  4.444188363190804
The total strategy returns are  4.444188363190804
[*********************100%********************

In [5]:
# Convert the results in to dataframe for validation
Pair_Strategy = pd.DataFrame({'Stock_X':X,'Stock_Y':Y,'Lookback' :lookbackperiod,'Std dev':stdev_range,'Hedge_Ratio':Hedge_ratio,'ADF Tstat': ADF_Tstat,'Strategy Returns':Strat_Returns,'Max Drawdown': Drawdown})


In [6]:
# Sort the dataframe based on highest returns
Pair_Strategy.sort_values(by='Strategy Returns',ascending=False)


,Stock_X,Stock_Y,Lookback,Std dev,Hedge_Ratio,ADF Tstat,Strategy Returns,Max Drawdown
548,LTI.NS,TECHM.NS,10,1.0,0.245210,-1.790653,21.820915,-6.950147
452,TECHM.NS,LTI.NS,10,1.0,4.054203,-1.779989,21.780360,-6.920543
365,WIPRO.NS,LTI.NS,20,1.5,8.594524,-1.146837,21.225403,-8.117199
356,WIPRO.NS,LTI.NS,10,1.0,8.594524,-1.146837,21.024175,-8.117199
532,LTI.NS,WIPRO.NS,10,1.0,0.114350,-1.135442,20.447428,-8.095813
...,...,...,...,...,...,...,...,...
89,TCS.NS,MINDTREE.NS,15,1.5,1.025093,-1.380380,-22.919195,-25.588215
85,TCS.NS,MINDTREE.NS,10,1.5,1.025093,-1.380380,-23.169330,-27.078599
581,MINDTREE.NS,TCS.NS,10,1.5,0.933832,-1.365778,-23.330120,-27.136478
69,TCS.NS,LTI.NS,10,1.5,1.515608,-1.388292,-23.807899,-27.137946
